In [1]:
import os
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
tsp_data_txt = open('tsp_dataset.txt')

In [3]:
lines = tsp_data_txt.readlines()

In [4]:
tsp_data_txt = open('tsp_dataset.txt')
data = {}
lines = tsp_data_txt.readlines()
distance_matrix = np.zeros((len(lines),len(lines)))
for i,line in enumerate(lines):
    for j,element in enumerate((line.split('\n')[0].split('       '))):
        if(element == ''):
            continue
        
        distance_matrix[i][j-1] = int(element.strip())
data['distance_matrix']= distance_matrix
data['reward_matrix'] = [1 for i  in range(0,len(distance_matrix))]
data['depot'] = 0
data['n_state'] = sum(list(range(1,len(distance_matrix))))
data['n_action'] = len(distance_matrix)
data['visited_nodes'] = []


In [5]:
for i in range(0,len(data['distance_matrix'])):
    data['distance_matrix'][i][i] = 9999


In [6]:
def change_reward(envp):
    for i in range(0,envp['n_state']):
        for j in range(0,envp['n_action']):
            for k in range(0,envp['n_action']):
                if k==0:
                    env['P'][i][j][k][2]= 10
                    env['P'][i][j][k][3]= True
                    break
    return envp

In [7]:
def initialze_value(env):
    initial_P = 1/(env['n_action']-1)
    env['P'] = {}
    
    #P[state][action] = [(prob, next state, reward, done), ...]
    
    for i in range(0,env['n_state']):
        env['P'][i] ={}
        
        if(i==104):
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                initial_P = 1/(env['n_action']-1)
                for k in range(0,env['n_action']):
                    if (j == env['depot'] and k == env['depot']):
                        ini_temp = [1,k,10,True]
                        env['P'][i][j].append(ini_temp)
                        initial_P = 0
                        continue
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
        else:
            for j in range(0,env['n_action']):
                env['P'][i][j] = []
                for k in range(0,env['n_action']):
                    ini_temp = [initial_P,k,data['reward_matrix'][k],False]
                    env['P'][i][j].append(ini_temp)
            
            
    return env
    

In [8]:
env = initialze_value(data)

In [9]:
class SARSAAgent():
    def __init__(self,n_state,n_action,epsilon=1,alpha=0.1,gamma=0.99):
        self.n_state = n_state
        self.n_action= n_action
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma =gamma
        self.Q = np.zeros([n_state,n_action])
        
    def update_Q(self, state,action,reward,state_prime, action_prime,done):
        Q_old = self.Q[state,action]
        if done :
            td_target = reward
        else :
            td_target = self.gamma*self.Q[state_prime,action_prime]
        td_error = td_target - Q_old
        self.Q[state,action] = Q_old+self.alpha*td_error
        
        #choose E- greedy policy
    def update_epsilon(self, epsilon):
        self.epsilon = np.min([epsilon,1])

    def get_action(self,state):
        if np.random.uniform() < self.epsilon:
            action = np.random.randint(0,high=self.n_action)
        else:
            action = np.argmax(self.Q[state])
        return action
        

In [10]:
n_state = env['n_state']
n_action = env['n_action']
agent = SARSAAgent(n_state,n_action,epsilon=1.0,alpha=0.1,gamma=0.999)

In [11]:
n_episiode = 100
ini_env = env
for e_idx in range(n_episiode):
    env['visited_nodes'] = []
    env = ini_env
    state = env['depot']
    env['visited_nodes'].append(state)
    for s in range(0,env['n_state']-1):
        for vn in env['visited_nodes']:
            env['P'][s][vn][vn] = [0,vn,-1,False]
    action = agent.get_action(state)
    done = False
    while not done:
        if len(env['visited_nodes'])==15:
            env['visited_nodes'] = []
            break
        prob,state_prime, reward,done  = env['P'][state][action][action]
        action_prime = agent.get_action(state_prime)
        agent.update_Q(state, action, reward, state_prime,action_prime, done)
        state = state_prime
        action = action_prime
        
        if action  not in env['visited_nodes']:
            env['visited_nodes'].append(action)
        
    # agent.update_epsilon(1000/(e_idx+1)) # reduce randomness
    agent.update_epsilon(1000/(e_idx+1)) # reduce update rate
print("sarsa done.")

sarsa done.


In [16]:
# Test
gamma = 0.99
env = ini_env
ret = 0
state = env['depot']
visited_list = []
for tick in range(1000):
    print("\n tick:[{}]".format(tick))
    print(visited_list)
    action = agent.get_action(state) # select action
    if action not in  visited_list:
        visited_list.append(action)
    _,next_obs,reward,done = env['P'][state][action][action]
    obs = next_obs+state
    ret = reward + gamma*ret 
    state = next_obs
    
    if len(visited_list)== 15: break
print ("Return is [{:.3f}]".format(ret))


 tick:[0]
[]

 tick:[1]
[1]

 tick:[2]
[1, 11]

 tick:[3]
[1, 11, 7]

 tick:[4]
[1, 11, 7, 4]

 tick:[5]
[1, 11, 7, 4, 10]

 tick:[6]
[1, 11, 7, 4, 10, 8]

 tick:[7]
[1, 11, 7, 4, 10, 8]

 tick:[8]
[1, 11, 7, 4, 10, 8, 14]

 tick:[9]
[1, 11, 7, 4, 10, 8, 14, 6]

 tick:[10]
[1, 11, 7, 4, 10, 8, 14, 6, 0]

 tick:[11]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5]

 tick:[12]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[13]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[14]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[15]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[16]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[17]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[18]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[19]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9]

 tick:[20]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13]

 tick:[21]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13]

 tick:[22]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13]

 tick:[23]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13]

 tick:[24]
[1, 11, 7, 4, 10, 8, 14, 6, 0, 5

In [18]:
visited_list

[1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13, 12, 3, 2]

In [14]:
def get_reward(visit_list):
    depot =0
    distance = (data['distance_matrix'])[0][visit_list[0]]
    prev_cite = visit_list[0]
    for i in range(1,14):
        distance+= (data['distance_matrix'])[prev_cite][visit_list[i]]
        prev_cite =visit_list[i]

    distance+= (data['distance_matrix'])[prev_cite][visit_list[14]]
    return distance

In [20]:
get_reward([1, 11, 7, 4, 10, 8, 14, 6, 0, 5, 9, 13, 12, 3, 2,0])

712.0